In [ ]:
import json
import spotipy
import time
import pandas as pd
import numpy as np
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import datetime
import re
import pprint

username = 'key'
scope =  'key'
client_id = 'key'
client_secret = 'key'
redirect_uri = 'http://localhost:8888'

token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
now = datetime.datetime.now()
date = now.day 

############ get daily new released track ################3


sp = spotipy.Spotify(auth=token)
new_release_data=[]

response = sp.new_releases()
while response :
    next_album = response['albums']
    albums = response['albums']['items'][0]
    #album_name  = response['albums']['items'][0]['name']
    album_id =  response['albums']['items'][0]['id']
    song_name = albums['name'] 
    artist = albums['artists'][0]['name']
    artist_id = response['albums']['items'][0]['artists'][0]['id']
    market = response['albums']['items'][0]['available_markets']
    new_release_data.append([song_name,artist,artist_id,album_id,market])
    if next_album['next']:
        response = sp.next(next_album)
    else:
        response = None
        

new_release_data2 = pd.DataFrame(new_release_data)
new_release_data2.columns = ['song_name','artist','artist_id','album_id','available_market']
#print(new_release_data2)


filename = "new_release_%s.csv" % date
new_release_data2.to_csv(filename)
print('new release file completed')
######### basing on new release track, get artists details ###################33

token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
spotify = spotipy.Spotify(auth=token)

artist_details = []
for i in new_release_data2['artist_id']:
    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
    spotify = spotipy.Spotify(auth=token)

    artist = spotify.artist(i)
    related = sp.artist_related_artists(i)
    related_artist = []
    for j in range(len(related['artists'])):
        related_artist.append([related['artists'][j]['name']])
    name = artist['name']
    artist_id = i
    follower = artist['followers']
    follower2 = follower['total']
    genre = artist['genres']
    spotify_id = artist['id']
    image = artist['images']
    popularity = artist['popularity']
    artist_details.append([artist_id,name,follower2,genre,spotify_id,image,popularity,related_artist])

artist_details2 = pd.DataFrame(artist_details)
artist_details2.columns = ['artist_id','name','follower','genre','spotify_id','image','popularity','related_artist']

filename = "artists_detail_%s.csv" % date
artist_details2.to_csv(filename)
print('artist detail file completed')
######### basing on new release artist id get all related album details ################

token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
spotify = spotipy.Spotify(auth=token)

album_list=[]
for i in new_release_data2['artist_id']:
    albums = []
    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
    spotify = spotipy.Spotify(auth=token)
    results = sp.artist_albums(i, album_type='album')
    albums.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    seen = set() # to avoid dups
    albums.sort(key=lambda album:album['name'].lower())
    for album in albums:
        name = album['name']
        album_id = album['id']
        if name not in seen:
            album_list.append([ name, album_id,i])
            seen.add(name)

album_list2=pd.DataFrame(album_list)
album_list2.columns=['album_name','album_id','artist_id']

token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
sp = spotipy.Spotify(auth=token)


album_details=[]
for i in album_list2['album_id']:
    
    #album_id = i
    #album_id = re.search('album\:(.+)', album_id).group(1)
    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
    spotify = spotipy.Spotify(auth=token)
    detail1 = sp.album(i)
    artist_id = detail1['artists'][0]['id']
    album_name = detail1['name']
    #print(artist_id)
    #artist_name = sp.artist(artist_id)
    #artist_name2 = artist_name['name']
    genres = detail1['genres']
    popularity = detail1['popularity']
    release_date = detail1['release_date']
    image = detail1['images']
    tracks = detail1['tracks']
    album_details.append([artist_id,
                          album_name,
                          i,market,genres,popularity,release_date,image,tracks])

album_details=pd.DataFrame(album_details)
album_details.columns=['Artist_id','Album_name','Album_id','available_markets','genres','popularity',
                   'release_date','image','tracks']

filename = "album_details_%s.csv" % date
album_details.to_csv(filename)
print('artists album detais file completed')

###### get artists' top songs ######
token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
sp = spotipy.Spotify(auth=token)


tids = []
for artist in new_release_data2['artist_id']:
    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
    sp = spotipy.Spotify(auth=token)
    results = sp.artist_top_tracks(artist)
    
    #print(results)
    for i in (results['tracks']):
        #print(i['name'])
        tids.append([i['name'],i['id'],artist])
#print(tids)
tids = pd.DataFrame(tids)
tids.columns=['song_name','song_id','artist_id']

############ get artists song details ################
details = []
#available_market=[]

username = 'yrzhou'
scope =  'user-library-read'
client_id = '8890b85e0aaa472ab0a12ef15849ecff'
client_secret = '44f08fc8b0a74382b607ca1364b1d984'
redirect_uri = 'http://localhost:8888'

for song in tids['song_id']:
    track = sp.track(song)
    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
    sp = spotipy.Spotify(auth=token)
    features = sp.audio_features(song)
    #features[0]['danceability'] = 0 if features is None
    for i in track['artists']:
    #for j in i['Artist']:
        #print(song)
        danceability=0 if features == [None] else features[0]['danceability']
        energy=0 if features == [None] else features[0]['energy']
        key = 0 if features ==[None] else features[0]['key']
        loudness= 0 if features == [None] else features[0]['loudness']
        mode = 0 if features == [None] else features[0]['mode']              
        speechiness= 0 if features == [None] else features[0]['speechiness']
        acousticness= 0 if features == [None] else features[0]['acousticness']
        instrumentalness = 0 if  features == [None] else features[0]['instrumentalness']
        liveness = 0 if  features == [None] else features[0]['liveness']
        valence = 0 if  features == [None] else features[0]['valence']
        tempo = 0 if features == [None] else features[0]['tempo']
        type2 = 0 if features == [None] else features[0]['type']
        duration_ms = 0 if  features == [None] else features[0]['duration_ms']
        details.append([song,i['name'],i['id'],[re.search('album\:(.+)', track['album']['uri']).group(1)],
                        danceability,
                        energy,
                        key,
                        loudness,
                        mode,
                        speechiness,
                        acousticness,
                        instrumentalness,
                        liveness,
                        valence,
                        tempo,
                        type2,
                        duration_ms, 
                        track['available_markets'],track['disc_number'],track['track_number']])
        #danceability.append(features[0]['danceability'])
        #for j in track['available_markets']:
            #details.append([j])
            #for k in track['popularity']:
             #   details.append(k)

#for i in track['available_markets']:
 #   available_market.append(i)
    
#print(type(track))    
datails = pd.DataFrame(details)
details2=pd.DataFrame(details)
details2.columns=['song_id','Artist_name','Artist_id','album_id','danceability','energy','key','loudness','mode','speechiness',
                 'acousticness','instrumentalness','liveness','valence','tempo','type','duration_ms','available_markets',
                 'Disc_Number','track_number']
#print(details2)
#print(available_market)

#details2.to_csv('songs.csv')
filename = "artists_top_song_detail_%s.csv" % date
details2.to_csv(filename)
print('artists top songs detail file completed')

############ basing on newly released songs, get song details ###############
token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
sp = spotipy.Spotify(auth=token)
#song_id = []

details = []
song_list = []
for i in new_release_data2['song_name']:
    track=(sp.search(q=i,type = 'track'))
    song_id = track['tracks']['items'][0]['id']
    song_name = track['tracks']['items'][0]['name']
    song_list.append([song_id,song_name])
    
song_list=pd.DataFrame(song_list)
song_list.columns=['song_id','song_name']

for inx, song in enumerate(song_list['song_id']):
    track = sp.track(song)
    features = sp.audio_features(song)
    song_name = song_list['song_name'].loc[inx]
    for i in track['artists']:
        token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
        sp = spotipy.Spotify(auth=token)
        danceability=0 if features == [None] else features[0]['danceability']
        energy=0 if features == [None] else features[0]['energy']
        key = 0 if features ==[None] else features[0]['key']
        loudness= 0 if features == [None] else features[0]['loudness']
        mode = 0 if features == [None] else features[0]['mode']              
        speechiness= 0 if features == [None] else features[0]['speechiness']
        acousticness= 0 if features == [None] else features[0]['acousticness']
        instrumentalness = 0 if  features == [None] else features[0]['instrumentalness']
        liveness = 0 if  features == [None] else features[0]['liveness']
        valence = 0 if  features == [None] else features[0]['valence']
        tempo = 0 if features == [None] else features[0]['tempo']
        type2 = 0 if features == [None] else features[0]['type']
        duration_ms = 0 if  features == [None] else features[0]['duration_ms']
        details.append([song,song_name,i['name'],i['id'],[re.search('album\:(.+)', track['album']['uri']).group(1)],
                        danceability,
                        energy,
                        key,
                        loudness,
                        mode,
                        speechiness,
                        acousticness,
                        instrumentalness,
                        liveness,
                        valence,
                        tempo,
                        type2,
                        duration_ms,
                        track['available_markets'],track['disc_number'],track['track_number']])

        #for j in track['available_markets']:
         #   details.append([j])
            #for k in track['popularity']:
             #   details.append(k)

#for i in track['available_markets']:
 #   available_market.append(i)
    
#print(type(track))    
datails = pd.DataFrame(details)
details3=pd.DataFrame(details)
details3.columns=['song_id','Song_name','Artist_name','Artist_id','album_id','danceability','energy','key','loudness','mode','speechiness',
                 'acousticness','instrumentalness','liveness','valence','tempo','type','duration_ms','available_markets',
                 'Disc_Number','track_number']

        #for j in track['available_markets']:
         #   details.append([j])
            #for k in track['popularity']:
             #   details.append(k)

#for i in track['available_markets']:
 #   available_market.append(i)
    
#print(type(track))    
datails = pd.DataFrame(details)
details3=pd.DataFrame(details)
details3.columns=['song_id','Song_name','Artist_name','Artist_id','album_id','danceability','energy','key','loudness','mode','speechiness',
                 'acousticness','instrumentalness','liveness','valence','tempo','type','duration_ms','available_markets',
                 'Disc_Number','track_number']


filename = "new_release_song_detail_%s.csv" % date
details3.to_csv(filename)

print('new released song detail file completed')